# Recategorize score
 Rescore model probabilities using Buckets from train. Saves results to specified database table

## Load libraries

In [2]:
pip install tabgan==1.3.3

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install lightgbm

In [4]:
pip install scikit-plot

  Using cached scikit_plot-0.3.7-py3-none-any.whl.metadata (7.1 kB)
Using cached scikit_plot-0.3.7-py3-none-any.whl (33 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install scipy==1.11.4

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install --upgrade pandas "dask[complete]"

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install scikit-learn==1.1.3

  Using cached scikit_learn-1.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
Using cached scikit_learn-1.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (30.5 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.4.2
    Uninstalling scikit-learn-1.4.2:
      Successfully uninstalled scikit-learn-1.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data-insights 0.3.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [8]:
## librerias requeridas
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import calendar
import re
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from scikitplot.metrics import plot_roc
from sklearn.metrics import roc_auc_score
from scikitplot.metrics import plot_lift_curve
from scikitplot.helpers import binary_ks_curve 
from scikitplot.metrics import plot_ks_statistic
from scikitplot.helpers import cumulative_gain_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from lightgbm import LGBMClassifier
from datetime import datetime, timedelta

import pyarrow as pa
import pyarrow.parquet as pq

In [9]:
import sklearn
print(sklearn.__version__)

1.1.3


In [10]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.memory_usage', True)

## Load model and data to evaluate

Load variables names from file

In [11]:
import os
os.getcwd()

'/root/viamericas-dev/Analytics/5.Fraud'

In [12]:
import os
import joblib
# load model
modelo = joblib.load("ModeloAWSRetrain.pkl") 

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.4.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
modelo

LGBMClassifier(colsample_bytree=0.84, importance_type='gini',
               learning_rate=0.07, max_depth=20, min_child_weight=0.0026,
               n_estimators=520, num_leaves=30, objective='binary',
               scale_pos_weight=0.003, subsample=1)

In [14]:
#2da prueba
var_input = ['branch_minutes_since_last_transaction',
    'branch_trans_3m',  'branch_has_fraud',  
    'branch_trans_40min', 'branch_trans_10min', 'cash_pick_up_40min', 
    'location_nro_fraud', 'sender_trans_3m', 
    'sender_nro_fraud','a_isMexico', 
    'a_sender_branch_state', 'a_var_range_hist', 
    'a_branch_fraud', 'a_location_fraud',
    'a_isCashPick', 'a_isBankDep', 
    'sender_days_to_last_transaction_more7m', 
    'receiver_transaction_count','a_sender_sending_days',
    'branch_working_days', 'a_net_amount_receiver', 
    'sender_minutes_since_last_transaction_2days', 
    'sender_days_to_last_transaction_365', 
    'sender_days_to_last_transaction_7m', 'a_hour_receiver']

In [15]:
s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Data4Model/TargetTodosRetrain"
#tes = pd.read_parquet(f"s3://{s3_url}/Data2024/Data2024.parquet")

tes = pd.read_parquet(f"s3://{s3_url}/Test/Test.parquet")

## Variable list for input
X = tes[var_input]
Y = tes[['target']]

dias_target = ['day_name_receiver_0', 'day_name_receiver_1',
       #'day_name_receiver_2', 'day_name_receiver_3',
       #'day_name_receiver_4', 'day_name_receiver_5',
       'day_name_receiver_6','target']

testeo =  tes.copy()
Y_target = pd.DataFrame(tes[dias_target])



In [16]:
#s3_url = f"viamericas-datalake-dev-us-east-1-283731589572-raw/FraudModel/Data4Model/TargetTodos"
#tes = pd.read_parquet(f"s3://{s3_url}/Data2024/Data2024.parquet")

#testeo = pd.concat([testeo, tes], axis = 0, ignore_index=True)
#Y_target = pd.DataFrame(pd.concat([Y_target, tes[dias_target]], ignore_index=True)) 

## Apply model

#Rename variables
testeo2.rename(columns={'01_isMexico':'a_isMexico',
    '01_sender_branch_state':'a_sender_branch_state', 
    '01_var_range_hist':'a_var_range_hist', 
    '01_branch_fraud':'a_branch_fraud', 
    '01_location_fraud':'a_location_fraud',
    '01_isCashPick':'a_isCashPick', 
    '01_isBankDep':'a_isBankDep', 
    '01_sender_sending_days':'a_sender_sending_days',
    '01_net_amount_receiver':'a_net_amount_receiver' }, inplace=True)

In [17]:
testeo2 = testeo[var_input]

In [18]:
# Prediciendo la variable objetivo para el conjunto de datos de prueba 'X_TESTEO' utilizando el clasificador LightGBM
testeo_data_pred_rf = modelo.predict(testeo2)

# Prediciendo las probabilidades de clase para cada observación en 'X_TESTEO' utilizando el clasificador LightGBM
probab_rf = modelo.predict_proba(testeo2)

# Eliminando la primera columna (se asume que son las probabilidades de la clase negativa) de las probabilidades predichas
# Se asume que la segunda columna contiene las probabilidades para la clase positiva
score_rf = np.delete(probab_rf, np.s_[0], axis=1)


In [19]:

# Creando una copia del DataFrame 'Y_TESTEO' para preservar los datos originales
Y_c = Y_target.copy()

# Agregando una nueva columna 'preds_rf' al DataFrame copiado con las predicciones del clasificador LightGBM
Y_c['preds_rf'] = testeo_data_pred_rf

# Agregando una nueva columna 'score_rf' al DataFrame copiado con las probabilidades de la clase positiva
Y_c['score_rf'] = score_rf

In [20]:
pd.DataFrame(confusion_matrix(Y_c['target'], Y_c['preds_rf']))

,0,1
0,5013233,0
1,45,2


## Transformo los datos para subirlos a Athena

In [21]:
Buckets = pd.read_csv('./BucketsModeloAWSRetrain.csv')

In [22]:
# Create bins from reference dataframe
pd.options.display.float_format = "{:,.15f}".format
# Convert data types of 'Min_Value' and 'Max_Value' to numeric
Buckets['min'] = pd.to_numeric(Buckets['min'])
Buckets['max'] = pd.to_numeric(Buckets['max'])
Y_c['score_rf'] = pd.to_numeric(Y_c['score_rf'])

#bins = np.sort(np.append(Buckets['min'], Buckets['max'].iloc[-1]))
bins = np.sort(np.append(Buckets['min'], np.inf))  

labels = np.sort(Buckets['final_score'])
#labels = np.sort(1000-Buckets['FRAUD_DECILE'])

# Recode scores using pd.cut()
Y_c['score_AWS'] = pd.cut(Y_c['score_rf'], bins=bins, labels=labels) ##, include_lowest=True, right=False)
#Y_c['Recode'] = Y_c['Recode'].astype('Int32')

Y_c.head()

,day_name_receiver_0,day_name_receiver_1,day_name_receiver_6,target,preds_rf,score_rf,score_AWS
0,True,False,False,False,False,0.000000000878673,0.883000000000000
1,False,False,True,False,False,0.000000000207147,0.751000000000000
2,False,False,False,False,False,0.000000000021833,0.419000000000000
3,False,False,False,False,False,0.000000000003359,0.182000000000000
4,False,False,False,False,False,0.000000000026953,0.451000000000000


In [23]:
Y_c['pred_corte_01'] = Y_c['score_AWS']>0.998 

In [24]:
Y_c['pred_corte_01'].value_counts()

pred_corte_01
False    5010801
True        2479
Name: count, dtype: int64

In [25]:
#See false positives by month
tes_corte = testeo[Y_c['pred_corte_01']==True]
Y_c_corte = Y_c[Y_c['pred_corte_01']==True]

pd.crosstab(tes_corte['date_receiver'].dt.month, Y_c_corte['target'])

target,False,True
date_receiver,,
6,1693,16
7,761,9


In [26]:
tes_corte[tes_corte['date_receiver'].dt.day==31]

,id_branch,id_receiver,date_receiver,id_location,id_payer,id_sender_global,net_amount_receiver,id_country_receiver_claim,id_state_receiver_claim,id_state,branch_working_days,sender_sending_days,sender_days_to_last_transaction,id_country,fraud_classification,sender_minutes_since_last_transaction,branch_minutes_since_last_transaction,sender_days_since_last_transaction,branch_trans_3m,sender_state,branch_has_fraud,receiver_has_fraud,branch_trans_40min,branch_trans_10min,cash_pick_up_40min,location_nro_fraud,sender_trans_3m,range_hist,sender_nro_fraud,receiver_transaction_count,mes,hour_receiver,a_isMexico,a_sender_branch_state,a_var_range_hist,a_receiver_fraud,a_branch_fraud,a_location_fraud,a_sender_fraud,a_isCashPick,a_isBankDep,sender_days_to_last_transaction_365,sender_days_to_last_transaction_more365,sender_days_to_last_transaction_7m,sender_days_to_last_transaction_more7m,sender_minutes_since_last_transaction_2days,sender_minutes_since_last_transaction_1day,a_net_amount_receiver,a_receiver_transaction_count,a_sender_sending_days,a_sender_minutes_since_last_transaction_2days,a_sender_minutes_since_last_transaction_1day,a_sender_days_to_last_transaction_365,a_sender_days_to_last_transaction_7m,a_hour_receiver,day_name_receiver_0,day_name_receiver_1,day_name_receiver_2,day_name_receiver_3,day_name_receiver_4,day_name_receiver_5,day_name_receiver_6,id_payout_A,id_payout_C,id_payout_D,id_payout_G,id_payout_M,id_payout_N,id_payout_O,id_payout_P,id_payout_S,id_payout_T,id_payout_X,targetR,targetP,target
2967636,A19658,23112,2024-07-31 14:48:54.243,IN0119,T057,7371658,2200,MEX,MEX8,IN,1483,194,54,USA,0,77723,76,53,"1,299.000000000000000",IN,3.000000000000000,NaN,NaN,NaN,NaN,3.000000000000000,1,"1,750.000000000000000",0,1,7,14,True,True,1.257142857142857,False,True,True,False,False,True,54,0,54,0,0,0,0.794223826714801,0.000000000000000,0.346428571428571,0.000000000000000,0.000000000000000,0.147945205479452,0.257142857142857,0.583333333333333,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
2994353,A21729,3289,2024-07-31 15:09:02.303,CA3984,T160,11543228,1500,MEX,MEX21,CA,1024,0,0,USA,0,0,88,0,317.000000000000000,CA,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1,7,15,True,True,NaN,False,False,False,False,False,True,0,0,0,0,0,0,0.541516245487365,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.625000000000000,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
3145579,A24662,10792,2024-07-31 16:52:06.603,NY1450,T232,10874667,1750,MEX,MEX22,NY,418,181,181,USA,0,260576,17,180,"2,580.000000000000000",NY,NaN,NaN,1.000000000000000,NaN,NaN,NaN,0,NaN,0,1,7,16,True,True,NaN,False,False,False,False,True,False,181,0,181,0,0,0,0.631768953068592,0.000000000000000,0.323214285714286,0.000000000000000,0.000000000000000,0.495890410958904,0.861904761904762,0.666666666666667,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
3159174,A25084,982,2024-07-31 18:02:11.870,GA1260,T160,10611775,2800,MEX,MEX25,GA,355,0,0,USA,0,0,1404,0,250.000000000000000,GA,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1,7,18,True,True,NaN,False,False,False,False,False,True,0,0,0,0,0,0,1.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.750000000000000,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False
3312365,A18743,20417,2024-07-31 18:24:31.137,NM0041,T232,7911861,985,MEX,MEX15,NM,1693,0,0,USA,0,0,44,0,"1,423.000000000000000",NM,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,1,7,18,True,True,NaN,False,False,False,False,False,True,0,0,0,0,0,0,0.355595667870036,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.000000000000000,0.750000000000000,False,Fals

In [27]:
df = pd.concat([testeo[['id_branch','id_receiver']], Y_c[['score_AWS','pred_corte_01']]], axis=1) 
df = df.rename(columns={"pred_corte_01": "was_fraud_AWS"})

## Guardo en Athena

In [28]:
pip install awswrangler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.7/381.7 kB 5.0 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [29]:
df

,id_branch,id_receiver,score_AWS,was_fraud_AWS
0,A26003,4316,0.883000000000000,False
1,A10051,101467,0.751000000000000,False
2,A22458,11617,0.419000000000000,False
3,A16988,135266,0.182000000000000,False
4,A26256,2581,0.451000000000000,False
...,...,...,...,...
5013275,A05164,357697,0.780000000000000,False
5013276,A22856,15845,0.934000000000000,False
5013277,A22119,32660,0.688000000000000,False
5013278,A26012,711,0.641000000000000,False


In [30]:
import awswrangler as wr
import pandas as pd
import boto3
import time
from time import sleep

# Reemplazar aca por el dataframe a guardar
df = df

wr.s3.to_parquet(
    df=df,
    path='s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/ml_score_aws/',
    dataset=True,
    mode="overwrite"
)

#lambda_crawler
def wait_for_crawler_completion(crawler_name: str, poll_interval: int = 15) -> str:
	"""
	Waits until Glue crawler completes and
	returns the status of the latest crawl run.
	Raises AirflowException if the crawler fails or is cancelled.
	:param crawler_name: unique crawler name per AWS account
	:param poll_interval: Time (in seconds) to wait between two consecutive calls to check crawler status
	:return: Crawler's status
	"""
	client = boto3.client('glue')
	response = client.start_crawler(Name=crawler_name)
	failed_status = ['FAILED', 'CANCELLED']
	sleep(5)

	while True:
		crawler = client.get_crawler(Name=crawler_name)
		crawler_state = crawler['Crawler']['State']
		if 'LastCrawl' in crawler['Crawler']:
			if crawler_state == 'STOPPING' or crawler_state == 'READY':
				print("State: %s", crawler_state)
				print("crawler_config: %s", crawler)
				crawler_status = crawler['Crawler']['LastCrawl']['Status']
				if crawler_status in failed_status:
						raise Exception(f"Status: {crawler_status}")
				metrics = client.get_crawler_metrics(CrawlerNameList=[crawler_name])['CrawlerMetricsList'][0]
				print("Status: %s", crawler_status)
				print("Last Runtime Duration (seconds): %s", metrics['LastRuntimeSeconds'])
				print("Median Runtime Duration (seconds): %s", metrics['MedianRuntimeSeconds'])
				print("Tables Created: %s", metrics['TablesCreated'])
				print("Tables Updated: %s", metrics['TablesUpdated'])
				print("Tables Deleted: %s", metrics['TablesDeleted'])

				return crawler_status

			else:
					print("Polling for AWS Glue crawler: %s ", crawler_name)
					print("State: %s", crawler_state)

					metrics = client.get_crawler_metrics(CrawlerNameList=[crawler_name])[
							'CrawlerMetricsList'
					][0]
					time_left = int(metrics['TimeLeftSeconds'])

					if time_left > 0:
							print("Estimated Time Left (seconds): %s", time_left)
					else:
							print("Crawler should finish soon")

					time.sleep(poll_interval)
		else:
			if crawler_state == 'READY':
					print("State: %s", crawler_state)
					print("crawler_config: %s", crawler)
					crawler_status = crawler['Crawler']['LastCrawl']['Status']
					if crawler_status in failed_status:
							raise Exception(f"Status: {crawler_status}")
					metrics = client.get_crawler_metrics(CrawlerNameList=[crawler_name])['CrawlerMetricsList'][0]
					print("Status: %s", crawler_status)
					print("Last Runtime Duration (seconds): %s", metrics['LastRuntimeSeconds'])
					print("Median Runtime Duration (seconds): %s", metrics['MedianRuntimeSeconds'])
					print("Tables Created: %s", metrics['TablesCreated'])
					print("Tables Updated: %s", metrics['TablesUpdated'])
					print("Tables Deleted: %s", metrics['TablesDeleted'])

					return crawler_status

			else:
					print("Polling for AWS Glue crawler: %s ", crawler_name)
					print("State: %s", crawler_state)

					metrics = client.get_crawler_metrics(CrawlerNameList=[crawler_name])[
							'CrawlerMetricsList'
					][0]
					time_left = int(metrics['TimeLeftSeconds'])

					if time_left > 0:
							print("Estimated Time Left (seconds): %s", time_left)
					else:
							print("Crawler should finish soon")

					time.sleep(poll_interval)


CrawlerName = 'crw_fraud_model_input'
crawler_to_run = f'{CrawlerName}'
print('Running Crawler: ', crawler_to_run)
response = wait_for_crawler_completion(crawler_name=crawler_to_run)
print('Finish with status: ', response)

Running Crawler:  crw_fraud_model_input
Polling for AWS Glue crawler: %s  crw_fraud_model_input
State: %s RUNNING
Crawler should finish soon
Polling for AWS Glue crawler: %s  crw_fraud_model_input
State: %s RUNNING
Crawler should finish soon
Polling for AWS Glue crawler: %s  crw_fraud_model_input
State: %s RUNNING
Crawler should finish soon
Polling for AWS Glue crawler: %s  crw_fraud_model_input
State: %s RUNNING
Estimated Time Left (seconds): %s 15
State: %s STOPPING
crawler_config: %s {'Crawler': {'Name': 'crw_fraud_model_input', 'Role': 'sdlf-lakeformation-admin', 'Targets': {'S3Targets': [{'Path': 's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/ml_score_aws/', 'Exclusions': []}], 'JdbcTargets': [], 'MongoDBTargets': [], 'DynamoDBTargets': [], 'CatalogTargets': [], 'DeltaTargets': [], 'IcebergTargets': [], 'HudiTargets': []}, 'DatabaseName': 'analytics', 'Classifiers': [], 'RecrawlPolicy': {'RecrawlBehavior': 'CRAWL_EVERYTHING'}, 'SchemaChangePolicy': {'UpdateBehavior